# This notebook can be ran as slides from your browser by:
* Downloading the notebook from github
* Moving to the folder that has the notebook in terminal/console i.e. 'cd ~/Downloads'
* Enter: 'jupyter nbconvert Data_verify_clean.ipynb --to slides --post serve'

## Importing the early dependancies and data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# %load get_data.py
def get_data() :
    
    import pandas as pd
    
    # rating_update is the original data 'beer_review.csv' with beer_style updated
    # The updates change beer_style to match the styles that are given in 'beer_description.csv'

    csv_beer = pd.read_csv("/home/grimoire/Projects/BeerRatings/beer_reviews.csv")
    beer_ratings = pd.DataFrame(csv_beer)
    
    return beer_ratings


In [3]:
beer_ratings = get_data()

# The purpose of this notebook will be to:
* Ameliorate, or remove bad values if necessary
* Verify all numeric fall into appropriate ranges (1-5) and/or non-negative

# Ameliorate (to make better or more tolerable) or remove bad values

# The hunt for bad review_profilename

In [4]:
beer_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


The RangeIndex gives use a total of 1,586,614 observations. 

We have 1,586,266 non-null object observations in review_profilename

In [5]:
profilename_array = \
    [name for name in beer_ratings.review_profilename.unique()]

Building an array of the unique/distinct profile names

Checking for 'nan', Not a Number or null, values in the array

In [6]:
print(True in [name == 'nan' for name in profilename_array])

False


In [7]:
print(True in [name == np.nan for name in profilename_array])

False


* Profile names should be type str. We're going to map the array so all values are type string. 
* After doing this we rechecked for 'nan' and find that there are nan values. 
In python nan can be a float value. So when checking for literal 'nan' it looked for an exact str match. 

Checking for these values as an array can save time before doing a large conversion over a dataframe.

In [8]:
print(True in [name == 'nan' for name in list(map(str, profilename_array))])

True


In [9]:
beer_ratings['review_profilename'] = \
    beer_ratings['review_profilename'].astype('str')
print(beer_ratings[beer_ratings['review_profilename'] == 'nan'].head(5))

       brewery_id                                brewery_name  review_time  \
8869          395  Bluegrass Brewing Co. - East St. Matthew's   1205005717   
22125        1199                    Founders Brewing Company   1231726538   
31822        1199                    Founders Brewing Company   1246501387   
33500        1199                    Founders Brewing Company   1238871081   
33678        1199                    Founders Brewing Company   1227224257   

       review_overall  review_aroma  review_appearance review_profilename  \
8869              4.0           4.0                3.5                nan   
22125             4.0           4.5                4.5                nan   
31822             4.0           4.0                4.0                nan   
33500             5.0           5.0                5.0                nan   
33678             5.0           5.0                5.0                nan   

                             beer_style  review_palate  review_taste

In [10]:
bad_ProfileIndices = \
    beer_ratings.loc[
        beer_ratings['review_profilename'] == 'nan',:]\
        ['review_profilename'].index

Collect the indices of the 'nan' entries in review_profilename

In [11]:
for index in bad_ProfileIndices :
    beer_ratings.loc[index,'review_profilename'] = 'UNKNOWN'

Set the values as 'OTHER'. In instances where we're not using the profile names we still have the rest of the data. In cases that we're going to use profile names we can find the old 'nan' values under 'OTHER'

In [12]:
print(beer_ratings[beer_ratings['review_profilename'] == 'UNKNOWN'].head(5))

       brewery_id                                brewery_name  review_time  \
8869          395  Bluegrass Brewing Co. - East St. Matthew's   1205005717   
22125        1199                    Founders Brewing Company   1231726538   
31822        1199                    Founders Brewing Company   1246501387   
33500        1199                    Founders Brewing Company   1238871081   
33678        1199                    Founders Brewing Company   1227224257   

       review_overall  review_aroma  review_appearance review_profilename  \
8869              4.0           4.0                3.5            UNKNOWN   
22125             4.0           4.5                4.5            UNKNOWN   
31822             4.0           4.0                4.0            UNKNOWN   
33500             5.0           5.0                5.0            UNKNOWN   
33678             5.0           5.0                5.0            UNKNOWN   

                             beer_style  review_palate  review_taste

In [13]:
beer_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586614 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


All of the review_profilename anomolies have been made tolerable. Onto the next section...

It should be mentioned that it may be possible to map a profile name to these 'OTHER' accounts. In this project we won't focus on this

# The hunt for bad brewery_name values

In [14]:
beer_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586614 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


## We'll follow the same procedure as we started with last time

In [15]:
brewery_name_asStr = list(map(str, beer_ratings.brewery_name.unique()))

In [16]:
print(True in [name == 'nan' for name in brewery_name_asStr])

True


In [17]:
beer_ratings['brewery_name'] = beer_ratings['brewery_name'].astype('str')

## Check what what brewery_id's we're dealing with

## We may be able to map good values to these bad values

In [18]:
beer_ratings[beer_ratings['brewery_name'] == 'nan'].brewery_id.unique()

array([1193,   27])

In [19]:
print(beer_ratings[beer_ratings['brewery_id'] == 27].brewery_name.unique())
print(beer_ratings[beer_ratings['brewery_id'] == 1193].brewery_name.unique())

['nan']
['nan']


### Checking on BeerAdvocate using those ID's doesn't fetch any brewery.

## We'll check by beer name next:

In [20]:
beer_ratings[beer_ratings['brewery_id'] == 27]['beer_name'].unique()

array(['Hard Hat American Beer', 'Side Pocket', 'Breakaway IPA',
       'Caboose Oatmeal Stout'], dtype=object)

In [21]:
beer_ratings[beer_ratings['brewery_id'] == 1193]['beer_name'].unique()

array(['Engel Tyrolian Bräu WRONG BREWERY SEE SCHWABISCH GMUND',
       'Engel Bock Dunkel WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Gold WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Landbier WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Keller Hell WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Aloisius - WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Keller Dunkel  WRONG BREWERY SEE CRAILSHEIMER'], dtype=object)

First we'll deal with id 1193. It provides us a note as to what breweries these should be. 
We'll start by removing the single 'Engel Tyrolian Bräu WRONG BREWERY SEE SCHWABISCH GMUND' entry from the list.

In [22]:
SCHWABISCH_GMUND = \
    beer_ratings[beer_ratings['beer_name']==\
                 'Engel Tyrolian Bräu WRONG BREWERY SEE SCHWABISCH GMUND'].index
print(beer_ratings[beer_ratings['beer_name']==\
                   'Engel Tyrolian Bräu WRONG BREWERY SEE SCHWABISCH GMUND'].index)

Int64Index([651565], dtype='int64')


Find where the entry is located in the dataframe and replace with good data

In [23]:
beer_ratings.loc[SCHWABISCH_GMUND, 'brewery_name'] = 'Schwabisch Gmund'
# pulled from beeradvocate.com/beer/profile/4216/
beer_ratings.loc[SCHWABISCH_GMUND, 'brewery_id'] = 23282
beer_ratings.loc[SCHWABISCH_GMUND, 'beer_name'] = 'Engel Tyrolian Bräu'

Verify if by using the old id. We should only see CRAILSHEIMER on the list now.

In [24]:
beer_ratings[beer_ratings['brewery_id'] == 1193]['beer_name'].unique()

array(['Engel Bock Dunkel WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Gold WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Landbier WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Keller Hell WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Aloisius - WRONG BREWERY SEE CRAILSHEIMER',
       'Engel Keller Dunkel  WRONG BREWERY SEE CRAILSHEIMER'], dtype=object)

In [25]:
CRAILSHEIMER_indices = beer_ratings[beer_ratings['brewery_id'] == 1193]['beer_name'].index
# Find where the indices of the CRAILSHEIMER brewery occur

In [26]:
for index in CRAILSHEIMER_indices:
    beer_ratings.loc[index, 'brewery_name'] = 'Crailsheimer'
    # pulled from beeradvocate.com/beer/profile/4216/
    beer_ratings.loc[index, 'brewery_id'] = 4216
    
# Replace bad values (brewery_name, brewery_id) with proper values

Lastly we'll deal with brewery_id 27. These are beers without any obvious brewery match. Similar to the profile names we'll change this brewery_name to 'UNKNOWN' so that it has a valid value and can be easily found/isolated if necessary.

In [27]:
# Find the indices of brewery_id 27
bad_BreweryIndices = beer_ratings[beer_ratings['brewery_id'] == 27]['beer_name'].index

# Using those indices change the brewery name to 'UNKNOWN'
for index in bad_BreweryIndices :
    beer_ratings.loc[index,'brewery_name'] = 'UNKNOWN'

In [28]:
beer_ratings[beer_ratings['brewery_name'] == 'nan'].brewery_id.unique()

array([], dtype=int64)

In [29]:
beer_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586614 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586614 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


## We verify that the values look good, and move to the next section.

In [30]:
# beer_ratings.to_csv('/home/grimoire/Projects/BeerRatings/rating_update.csv', index=False)

# The hunt for beer_abv null (bad) values

In [31]:
beer_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586614 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586614 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


## Alternative way to look for bad values

In [32]:
beer_ratings.beer_abv.unique()

array([  5.00000000e+00,   6.20000000e+00,   6.50000000e+00,
         7.70000000e+00,   4.70000000e+00,   3.50000000e+00,
         7.20000000e+00,   5.60000000e+00,   7.40000000e+00,
         5.50000000e+00,   5.80000000e+00,   8.10000000e+00,
         6.10000000e+00,              nan,   8.80000000e+00,
         5.70000000e+00,   7.00000000e+00,   6.30000000e+00,
         3.90000000e+00,   6.70000000e+00,   6.00000000e+00,
         6.80000000e+00,   5.40000000e+00,   9.50000000e+00,
         4.80000000e+00,   5.20000000e+00,   9.40000000e+00,
         7.60000000e+00,   8.50000000e+00,   4.00000000e+00,
         4.50000000e+00,   4.20000000e+00,   9.00000000e+00,
         8.00000000e+00,   4.10000000e+00,   1.04000000e+01,
         5.10000000e+00,   1.00000000e+01,   3.80000000e+00,
         7.10000000e+00,   4.60000000e+00,   7.50000000e+00,
         1.00000000e-01,   8.40000000e+00,   4.40000000e+00,
         4.90000000e+00,   2.20000000e+00,   2.80000000e+00,
         5.30000000e+00,

## Changing the values, same as above in dataframe astype manipulation, to do some comparisons

In [33]:
beer_ratings[beer_ratings['beer_abv'] == np.nan]

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid


In [34]:
beer_ratings['beer_abv'] = beer_ratings.beer_abv.values.astype('str')

## Building sets of:
* beer_names that have 'nan' values
* beer_names that don't have 'nan values

Comparing between the two sets will allow us to find beers that we can map their ABV to the 'nan' values

In [35]:
bad_ABV_names = set(beer_ratings[beer_ratings['beer_abv'] == 'nan']['beer_name'].unique())

In [36]:
good_ABV_names = set(beer_ratings[beer_ratings['beer_abv'] != 'nan']['beer_name'].unique())

In [37]:
len(good_ABV_names & bad_ABV_names)

1338

In [38]:
beer_name_wABV_list = list(good_ABV_names & bad_ABV_names)

In [39]:
beer_ratings.keys()

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')

In [50]:
beer_ABV_map = {}
ratings_wABV = beer_ratings[beer_ratings['beer_abv'] != 'nan']
#for name in beer_name_wABV_list :
    #beer_ABV_map[name] = 
#    print(ratings_wABV[ratings_wABV['beer_name'] == name].beer_abv.values.mean())

In [56]:
ratings_wo_ABV.keys()

Index(['brewery_id', 'brewery_name', 'review_time', 'review_overall',
       'review_aroma', 'review_appearance', 'review_profilename', 'beer_style',
       'review_palate', 'review_taste', 'beer_name', 'beer_abv',
       'beer_beerid'],
      dtype='object')

In [79]:
name = beer_name_wABV_list[np.random.randint(0,len(beer_name_wABV_list)-1)]

ratings_wABV = beer_ratings[beer_ratings['beer_abv'] != 'nan']
ratings_wo_ABV = beer_ratings[beer_ratings['beer_abv'] == 'nan']

print(ratings_wABV.loc[ratings_wABV['beer_name'] == name]['brewery_name'])
print(ratings_wo_ABV.loc[ratings_wo_ABV['beer_name'] == name]['brewery_name'])

244239         BJ's Restaurant And Brewhouse
252193           BJ's Restaurant & Brewhouse
255213           BJ's Restaurant & Brewhouse
255956           BJ's Restaurant & Brewhouse
256547           BJ's Restaurant & Brewhouse
256548           BJ's Restaurant & Brewhouse
256549           BJ's Restaurant & Brewhouse
256550           BJ's Restaurant & Brewhouse
256551           BJ's Restaurant & Brewhouse
256552           BJ's Restaurant & Brewhouse
256553           BJ's Restaurant & Brewhouse
256554           BJ's Restaurant & Brewhouse
256555           BJ's Restaurant & Brewhouse
256556           BJ's Restaurant & Brewhouse
256557           BJ's Restaurant & Brewhouse
256558           BJ's Restaurant & Brewhouse
256559           BJ's Restaurant & Brewhouse
256560           BJ's Restaurant & Brewhouse
256561           BJ's Restaurant & Brewhouse
256562           BJ's Restaurant & Brewhouse
256563           BJ's Restaurant & Brewhouse
256564           BJ's Restaurant & Brewhouse
256565    

In [80]:
# name = 'Tatonka Stout'

'Tatonka Stout'

In [42]:
beer_ABV_map

{}

In [43]:
len(beer_ratings[beer_ratings.beer_abv =='nan'].beer_name)

67785

In [44]:
len(bad_ABV_names)

14110

In [45]:
len(good_ABV_names - bad_ABV_names)

42747

There are a total of 67,785 observations in the beer_abv column that have 'nan' values.
Of these 67,785 observations there are 14,110 individual specific beers with 'nan' values in beer_abv